<a href="https://colab.research.google.com/github/hanbun-pom/siosio/blob/master/Yukapedia_Git%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GitHub参照版

不特定多数への公開は今の所ご遠慮下さい  
当ページ外で行われた操作についての責任は負いかねます

>～使い方～  
>①左上　PlayBackgroundを選択  
>②左端の▶を押して実行

2020/03/19 追加　完全一致検索  正規表現検索　動画URL生成  
2020/04/06 試作　GitHub版  

In [32]:
import pandas as pd
import datetime

#@markdown ---
名前 = ""  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
#@markdown ---
コメント = ""  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = False  #@param {type:"boolean"} 
#@markdown ---
t秒前から再生 = 30 #@param {type:"slider", min:0, max:60, step:5}
#@markdown ---
配信日_以降 = '2019-12-24' #@param {type:"date"}
配信日_以前 = '2040-4-1' #@param {type:"date"}　


try:
    df

except NameError:

    # 配信一覧　取得
    youtube = 'https://youtu.be/'
    master = 'https://raw.githubusercontent.com/hanbun-pom/siosio/master/'
    videos = pd.read_csv(master + 'videos.csv')

    # コメント一覧　作成
    df_list=[]
    for Id in videos['videoId']:
        tmp_df = pd.read_table(master + Id + '.tsv', header=None, names=['time','name','comment','videoId'],quotechar='"', quoting=3)
        tmp_df['videoId']= Id
        df_list.append(tmp_df)
    df = pd.concat(df_list, ignore_index=True)

    # 配信タイトル、公開時間
    df = df.merge(videos)
    df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date

    # 再生時間指定URL
    trans = str.maketrans({':': '',',': ''})
    s = df['time'].str.translate(trans).astype(int)
    s = s-40*(abs(s)//100)-2400*(abs(s)//10000)-t秒前から再生
    s = ((abs(s)+s)//2).astype(str)

    df['videoId'] = youtube + df['videoId'] + '?t=' + s + 's'

else:
    pass

# 名前、コメント検索
def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

df2 = df[name_condition & comment_condition].copy()

# 配信日指定
start = datetime.datetime.strptime(配信日_以降, '%Y-%m-%d').date()
end = datetime.datetime.strptime(配信日_以前, '%Y-%m-%d').date()
df3 = df2[df2['DateTime'] >= start]
df3 = df3[df3['DateTime'] <= end]

pd.set_option('display.max_rows', 10000)
df3

ERROR! Session/line number was not unique in database. History logging moved to new session 67


,time,name,comment,videoId,title,DateTime
0,-1:41,如水,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
1,-1:37,紫ユウキ,しおしおしてきた！,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
2,-1:37,暇つぶし,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
3,-1:34,si n,しおしおっ,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
4,-1:34,pv order,待機🌸,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
...,...,...,...,...,...,...
796060,2:05:39,はやち,ありがとうーー,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796061,2:05:39,チヒロ氏,お疲れ様でした！,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796062,2:05:40,ふむふむ,あかん涙出てきた,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796063,2:05:40,法条裕一,お疲れさま～,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24


In [31]:
#df2(df2['DateTime'] > 配信日1)

#df['DateTime'][1]
start = datetime.datetime.strptime(配信日1, '%Y-%m-%d').date()
end = datetime.datetime.strptime(配信日2, '%Y-%m-%d').date()

df3 = df2[df2['DateTime'] >= start]
df3 = df3[df3['DateTime'] <= end]
df3

ERROR! Session/line number was not unique in database. History logging moved to new session 65


,time,name,comment,videoId,title,DateTime
0,-1:41,如水,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
1,-1:37,紫ユウキ,しおしおしてきた！,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
2,-1:37,暇つぶし,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
3,-1:34,si n,しおしおっ,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
4,-1:34,pv order,待機🌸,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
...,...,...,...,...,...,...
796060,2:05:39,はやち,ありがとうーー,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796061,2:05:39,チヒロ氏,お疲れ様でした！,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796062,2:05:40,ふむふむ,あかん涙出てきた,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796063,2:05:40,法条裕一,お疲れさま～,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24


In [30]:
df3 = df2[df2['DateTime'] >= start]
df3

,time,name,comment,videoId,title,DateTime
0,-1:41,如水,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
1,-1:37,紫ユウキ,しおしおしてきた！,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
2,-1:37,暇つぶし,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
3,-1:34,si n,しおしおっ,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
4,-1:34,pv order,待機🌸,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
...,...,...,...,...,...,...
796060,2:05:39,はやち,ありがとうーー,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796061,2:05:39,チヒロ氏,お疲れ様でした！,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796062,2:05:40,ふむふむ,あかん涙出てきた,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796063,2:05:40,法条裕一,お疲れさま～,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
